This notebook aims to build a graph purely with KNN.

In [1]:
import numpy as np
from nuscenes.nuscenes import NuScenes

nusc = NuScenes(version='v1.0-mini', dataroot='/media/HDD2/Datasets/mini_nusc', verbose=True)

# nusc = NuScenes(version='v1.0-trainval', dataroot='/media/HDD2/Datasets/nuscenes2', verbose=True)

Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 1.717 seconds.
Reverse indexing ...
Done reverse indexing in 0.4 seconds.


In [2]:
from nuscenes.utils.splits import train,val
from nuscenes.utils.splits import create_splits_scenes

split = create_splits_scenes()
print(split.keys())
split_scene_list = []
for scene_name in split['mini_train']:
    for scene in nusc.scene:
        if scene['name']==scene_name:
            split_scene_list.append(scene)

dict_keys(['train', 'val', 'test', 'mini_train', 'mini_val', 'train_detect', 'train_track'])


In [3]:
print(split['mini_train'])
print(split['mini_val'])

['scene-0061', 'scene-0553', 'scene-0655', 'scene-0757', 'scene-0796', 'scene-1077', 'scene-1094', 'scene-1100']
['scene-0103', 'scene-0916']


In [4]:
#write a sequence 

sample_dict = {}

i = 0 
for scene in split_scene_list:
    last_sample_token =""
    sample_token = scene['first_sample_token']
    while(last_sample_token == ""):
        
        sample = nusc.get('sample', sample_token)
        sample_dict[i] = (scene['token'],sample["token"])
        i += 1
        sample_token = sample["next"]
        if(sample["token"]== scene['last_sample_token']):
            last_sample_token = scene['last_sample_token']

print (len(sample_dict))
print (sample_dict[0])
print (sample_dict[1])
print (sample_dict[2])

323
('cc8c0bf57f984915a77078b10eb33198', 'ca9a282c9e77460f8360f564131a8af5')
('cc8c0bf57f984915a77078b10eb33198', '39586f9d59004284a7114a68825e8eec')
('cc8c0bf57f984915a77078b10eb33198', '356d81f38dd9473ba590f39e266f54e5')


In [5]:
import torch
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.is_available())
    num_gpu = torch.cuda.device_count()
    print("num_gpu:",num_gpu)
    print("current_device: ",torch.cuda.current_device())
    print("---------------------------------------------------------------")
    for i in range(num_gpu):
        print(torch.cuda.get_device_name(i))
        # print(torch.cuda.device(i))
        print('Memory Usage:')
        print('Allocated:', round(torch.cuda.memory_allocated(i)/1024**3,1), 'GB', torch.cuda.memory_allocated(i),torch.cuda.memory_allocated(i)/1024**3)
        print('Cached:   ', round(torch.cuda.memory_reserved(i)/1024**3,1), 'GB', torch.cuda.memory_reserved(i),torch.cuda.memory_reserved(i)/1024**3)
        print("---------------------------------------------------------------")

Using device: cuda
True
num_gpu: 2
current_device:  0
---------------------------------------------------------------
GeForce RTX 3090
Memory Usage:
Allocated: 0.0 GB 0 0.0
Cached:    0.0 GB 0 0.0
---------------------------------------------------------------
GeForce RTX 2080
Memory Usage:
Allocated: 0.0 GB 0 0.0
Cached:    0.0 GB 0 0.0
---------------------------------------------------------------


In [6]:
from utils.nuscenes_helper_functions import transform_detections_lidar2world_frame, get_ego_pose, get_sensor_pose, get_sensor_2_ego_transformation_matrix, get_ego_2_world_transformation_matrix, get_sample_data_table, get_gt_sample_annotation_pose
from pyquaternion import Quaternion
from typing import List
scene_token0, sample_token0= sample_dict[0]
sensor = "LIDAR_TOP"
lidar_top_data_table = get_sample_data_table(nusc, sensor, sample_token0)
_, boxes, _= nusc.get_sample_data(lidar_top_data_table['token'], selected_anntokens=None, use_flat_vehicle_coordinates =False)

# sensor_2_ego_transformation_matrix = get_sensor_2_ego_transformation_matrix(nusc,sensor, sample_token=sample_token0, inverse=False)
# ego_2_world_transformation_matrix = get_ego_2_world_transformation_matrix(nusc,sensor, sample_token=sample_token0, inverse=False)

for box in boxes:
    sample_annotation_token = box.token
    translation:List[float] = box.center.tolist()
    orientation:Quaternion = box.orientation
    print('translation',translation)
    print('orientation',orientation)

    translation_sensor_from_ego_frame, rotation_sensor_from_ego_frame = \
        get_sensor_pose(nusc, sensor, sample_token0)
    ego_transl, ego_rot= get_ego_pose(nusc, sensor, sample_token0)
    
    box.rotate(rotation_sensor_from_ego_frame)
    box.translate(translation_sensor_from_ego_frame)

    transformed_orientaion = box.orientation
    transformed_translation = box.center.tolist()
    print('transformed_translation',transformed_translation)
    print('transformed_orientaion',transformed_orientaion)
    
    box.rotate(ego_rot)
    box.translate(ego_transl)
    
    transformed_orientaion = box.orientation
    transformed_translation = box.center.tolist()
    print('transformed_translation',transformed_translation)
    print('transformed_orientaion',transformed_orientaion)

    translation_world_frame, orientation_world_frame = get_gt_sample_annotation_pose(nusc, box.token)
    print('translation_world_frame',translation_world_frame)
    print('orientation_world_frame',orientation_world_frame)
    
    comparison_array = (np.asarray(transformed_translation) - np.asarray(translation_world_frame)) < 0.001
    assert comparison_array.all()
    absolute_distance = (orientation_world_frame.absolute_distance(orientation_world_frame,transformed_orientaion))
    assert (absolute_distance < 0.001)

    transform_detections_lidar2world_frame(nusc, translation, orientation, sample_token0, box.token)


translation [18.41438499820346, 59.516025131224765, 0.76963457443623]
orientation 0.009 +0.008i -0.017j +1.000k
transformed_translation [60.49822359052301, -18.289041315080507, 1.0589517145171399]
transformed_orientaion 0.712 +0.004i -0.011j +0.702k
transformed_translation [373.256, 1130.419, 0.8000000000000007]
transformed_orientaion 0.983 +0.000i -0.000j -0.183k
translation_world_frame [373.256, 1130.419, 0.8]
orientation_world_frame 0.983 +0.000i +0.000j -0.183k
translation [21.002107038611666, 36.06110848124014, -0.02614779773018483]
orientation -0.724 -0.018i +0.006j -0.690k
transformed_translation [37.03621862929551, -20.923088585290003, 0.8164477400559131]
transformed_orientaion -1.000 -0.011i +0.005j +0.023k
transformed_translation [378.888, 1153.348, 0.8650000000000001]
transformed_orientaion -0.553 -0.000i +0.000j +0.833k
translation_world_frame [378.888, 1153.348, 0.865]
orientation_world_frame -0.553 +0.000i +0.000j +0.833k
translation [37.351860758272906, 64.39733873917031